In [22]:
import numpy as np
import pandas as pd
from scipy.stats import linregress
import sqlite3
import json
import numpy as np
import os
import pickle
import shutil
import sqlite3
from collections import defaultdict
from datetime import datetime
from time import sleep
#import query
import logzero
import requests
#import secret
import warnings
from pandas.core.common import SettingWithCopyWarning
from logzero import logger
from pandas.tseries.offsets import MonthEnd
from requests import get
import calendar
import regex as re
import pandas as pd
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)





In [23]:
conn_proj = sqlite3.connect("../data/Berkeley_2.db")
cursor_proj = conn_proj.cursor()

query_g = """
SELECT *

FROM grouped_df

"""


g_df = pd.read_sql_query(query_g, conn_proj)

In [24]:


def get_slope(array):
    y = np.array(array)
    x = np.arange(len(y))
    slope, intercept, r_value, p_value, std_err = linregress(x,y)
    return slope

In [25]:
g_df = g_df.sort_values(by = 'month')

g_df['rolling_slope'] = g_df.groupby(
    'zipcode')['count_per_home'].rolling(window=6,min_periods=6).apply(
    get_slope, raw=False).reset_index(0, drop=True)


In [26]:
g_df.sort_values(by = ['zipcode','month']).head(10)

,month,zipcode,i_count,size,price,discount,p_type_31,p_type_32,p_type_33,p_type_37,...,p_type_92,p_type_86,med_income,med_home_value,home_count,pop_density,NREL_PSM_2019,cost_electricity,count_per_home,rolling_slope
0,3.0,01001,0.0,0.0,11644.84,1506.85,0,0,0,-1,...,0,0,58733.0,213000.0,7557.0,1466.0,4.575282,21.92,0.0,NaN
9750,6.0,01001,0.0,0.0,11519.12,2793.05,0,0,0,-1,...,0,0,58733.0,213000.0,7557.0,1466.0,4.575282,21.92,0.0,NaN
19500,7.0,01001,0.0,0.0,11477.38,3206.44,0,0,0,-1,...,0,0,58733.0,213000.0,7557.0,1466.0,4.575282,21.92,0.0,NaN
29250,8.0,01001,0.0,0.0,11435.74,3612.23,0,0,0,-1,...,0,0,58733.0,213000.0,7557.0,1466.0,4.575282,21.92,0.0,NaN
39000,9.0,01001,0.0,0.0,11394.17,4010.49,0,0,0,-1,...,0,0,58733.0,213000.0,7557.0,1466.0,4.575282,21.92,0.0,NaN
48750,10.0,01001,0.0,0.0,11352.70,4401.26,0,0,0,-1,...,0,0,58733.0,213000.0,7557.0,1466.0,4.575282,21.92,0.0,0.0
58500,11.0,01001,0.0,0.0,11311.30,4784.60,0,0,0,-1,...,0,0,58733.0,213000.0,7557.0,1466.0,4.575282,21.92,0.0,0.0
68250,12.0,01001,0.0,0.0,11270.00,5160.54,0,0,0,-1,...,0,0,58733.0,213000.0,7557.0,1466.0,4.575282,21.92,0.0,0.0
78000,13.0,01001,0.0,0.0,11228.78,5529.16,0,0,0,-1,...,0,0,58733.0,213000.0,7557.0,1466.0,4.575282,21.92,0.0,0.0
87750,14.0,01001,0.0,0.0,11187.65,5890.48,0,0,0,-1,...,0,0,58733.0,213000.0,7557.0,1466.0,4.575282,21.92,0.0,0.0


In [27]:
elec = pd.read_csv("../data/RPE.csv")
elec.sample()


,description,description.1,units,source key,Jan 2001,Feb 2001,Mar 2001,Apr 2001,May 2001,Jun 2001,...,Aug 2020,Sep 2020,Oct 2020,Nov 2020,Dec 2020,Jan 2021,Feb 2021,Mar 2021,Apr 2021,May 2021
24,Residential : Indiana,Residential : Indiana,cents per kilowatthour,ELEC.PRICE.IN-RES.M,6.08,6.67,6.73,7.1,7.3,7.21,...,12.32,13.04,13.65,13.23,12.69,12.25,12.38,13.15,13.93,14.2


In [28]:
elec['description'] = elec.description.apply(lambda x: x.split(': ')[-1])
elec.sample(5)

,description,description.1,units,source key,Jan 2001,Feb 2001,Mar 2001,Apr 2001,May 2001,Jun 2001,...,Aug 2020,Sep 2020,Oct 2020,Nov 2020,Dec 2020,Jan 2021,Feb 2021,Mar 2021,Apr 2021,May 2021
49,Mississippi,Residential : Mississippi,cents per kilowatthour,ELEC.PRICE.MS-RES.M,6.45,6.69,7.14,7.42,7.76,7.81,...,10.77,10.84,11.58,11.98,11.25,10.88,11.34,11.52,12.13,12.42
50,Tennessee,Residential : Tennessee,cents per kilowatthour,ELEC.PRICE.TN-RES.M,6.11,6.23,6.32,6.36,6.46,6.37,...,10.55,10.6,11.06,11.1,10.41,10.36,10.27,10.71,11.57,11.5
3,monthly,Average retail price of electricity : United S...,cents per kilowatthour,ELEC.PRICE.US-IND.M,4.73,4.8,4.86,4.87,5,5.23,...,7.07,7.01,6.71,6.48,6.4,6.35,8.15,7.01,6.77,6.65
53,Louisiana,Residential : Louisiana,cents per kilowatthour,ELEC.PRICE.LA-RES.M,7.97,8.52,9.17,8.64,8.33,8.8,...,9.41,9.76,10.28,9.94,9.54,9.58,10,10.16,11.04,11.68
44,Virginia,Residential : Virginia,cents per kilowatthour,ELEC.PRICE.VA-RES.M,6.84,7.24,7.49,7.62,8.17,8.46,...,12.34,12.1,12.49,11.65,11.21,11.05,11.39,11.71,12.87,12.25


In [29]:


with open("cfg.json", "r") as jin:
    cfg = json.load(jin)

# rewrite to ensure formatting
with open("cfg.json", "w") as jout:
    json.dump(cfg, jout, indent=4)


query_zip_state = """
SELECT *

FROM zipcode

LEFT JOIN state on zipcode.state_id == state.id

"""

conn_proj = sqlite3.connect(cfg["db_dir"] + cfg["db_dsire"])
cursor_proj = conn_proj.cursor()

zip_state_df = pd.read_sql_query(query_zip_state, conn_proj)
    
conn_proj.close

<function Connection.close>

In [30]:
zip_state_df['zipcode'] = zip_state_df['zipcode'].apply(lambda a: str(a).zfill(5))

In [31]:
zip_state_df.head()


,id,zipcode,city_id,state_id,county_id,latitude,longitude,id,abbreviation,name,is_territory
0,1,00501,1,37,1,41,-73,37,NY,New York,0
1,2,00544,1,37,1,41,-73,37,NY,New York,0
2,3,00601,2,45,2,18,-67,45,PR,Puerto Rico,1
3,4,00602,3,45,3,18,-67,45,PR,Puerto Rico,1
4,5,00603,4,45,4,18,-67,45,PR,Puerto Rico,1


In [32]:
zip_rates = zip_state_df.merge(elec, left_on = 'name', right_on = 'description', how = 'left')

In [33]:
zip_rates.columns

Index(['id', 'zipcode', 'city_id', 'state_id', 'county_id', 'latitude',
       'longitude', 'id', 'abbreviation', 'name',
       ...
       'Aug 2020', 'Sep 2020', 'Oct 2020', 'Nov 2020', 'Dec 2020', 'Jan 2021',
       'Feb 2021', 'Mar 2021', 'Apr 2021', 'May 2021'],
      dtype='object', length=260)

In [34]:
min_mo = datetime.strptime(cfg["min_date"],"%d-%b-%Y (%H:%M:%S.%f)") + MonthEnd(1)



In [35]:
import datetime as datetime
from pandas.tseries.offsets import MonthEnd

round((datetime.datetime.strptime('Oct 2020', '%b %Y') + MonthEnd(1) - min_mo) /np.timedelta64(1, 'M'),0)

273.0

In [36]:
ddict = {}

for col in zip_rates.columns:
    try:
        col_name = datetime.datetime.strptime(col, '%b %Y') + MonthEnd(1)
        month = int(round((col_name - min_mo)/np.timedelta64(1, 'M'),0))
        ddict[month] = dict(zip(zip_rates.zipcode,zip_rates[col]))
    except:
        pass
    
    

In [37]:
def price_map(x):
    try:
        m = x[0]
        z = x[1]
        
        return ddict[m][z]
    except:
        return np.nan

In [38]:
g_df['cost_electricity'] = [price_map(x) for x in zip(g_df.month,g_df.zipcode)]

In [39]:
g_df = g_df.dropna()

In [40]:
conn_proj = sqlite3.connect("../data/Berkeley_2.db")
cursor_proj = conn_proj.cursor()

g_df.to_sql("g_df2", conn_proj, if_exists="replace", index=False)

conn_proj.commit

<function Connection.commit>

In [41]:
conn_proj = sqlite3.connect("../data/Berkeley_2.db")
cursor_proj = conn_proj.cursor()

query_g = """
SELECT *

FROM g_df2

"""


g_df = pd.read_sql_query(query_g, conn_proj)
g_df['cost_electricity'] = g_df['cost_electricity'].astype(float)

In [42]:
g_df2 = g_df[['count_per_home','NREL_PSM_2019','cost_electricity',
              'price','p_type_31','p_type_32','p_type_33','p_type_37',
              'p_type_46','p_type_50','p_type_78','p_type_87','p_type_88','p_type_92','p_type_86']]

In [43]:

# + p_type_50 + p_type_32 + p_type_33 + 

In [44]:
g_df2.dtypes

count_per_home      float64
NREL_PSM_2019       float64
cost_electricity    float64
price               float64
p_type_31             int64
p_type_32             int64
p_type_33             int64
p_type_37             int64
p_type_46             int64
p_type_50             int64
p_type_78             int64
p_type_87             int64
p_type_88             int64
p_type_92             int64
p_type_86             int64
dtype: object

In [45]:
# print(ols_robust2_1.summary())


In [46]:
cols_eval = ['p_type_31','p_type_32', 'p_type_33', 'p_type_37',
             'p_type_46', 'p_type_50', 'p_type_78', 'p_type_87', 'p_type_92', 'p_type_86']

from causalinference import CausalModel

treatment_dict = {}

In [48]:
for col in cols_eval:
    in_df = g_df[g_df[col] != 0].reset_index()
    in_df[col] = in_df[col].replace(-1,0)
    
    Y_i = in_df.count_per_home.values
    df_EVAL = in_df.drop(columns = ['month','zipcode','home_count','i_count', 'count_per_home'])


    print(col + ' started')
    try:
        D_i = df_EVAL[col].values
        df_t = df_EVAL[['NREL_PSM_2019', 'cost_electricity','price']].to_numpy()
        model = CausalModel(Y = Y_i, #outcome 
                            D = D_i, #treatment
                            X = df_t)

        print('model complete')
        model.est_propensity() 
        
        print('propensity complete')
        model.est_via_matching()  
        print('matching complete')
        #model.estimates

        ate = round(model.estimates['matching']['ate'],5)
        
        treatment_dict[col] = ate
    except Exception as e:
        print(e)
        treatment_dict[col] = 'unknown'
    print(col + ' complete')

p_type_31 started
model complete
propensity complete
matching complete
p_type_31 complete
p_type_32 started
model complete
propensity complete
matching complete
p_type_32 complete
p_type_33 started
Too few control units: N_c < K+1
p_type_33 complete
p_type_37 started
model complete
propensity complete
matching complete
p_type_37 complete
p_type_46 started
model complete
propensity complete
matching complete
p_type_46 complete
p_type_50 started
Too few control units: N_c < K+1
p_type_50 complete
p_type_78 started
model complete
propensity complete
matching complete
p_type_78 complete
p_type_87 started
model complete
propensity complete
matching complete
p_type_87 complete
p_type_92 started
model complete
propensity complete
matching complete
p_type_92 complete
p_type_86 started
model complete
propensity complete
matching complete
p_type_86 complete


In [49]:
g_df3 = g_df[g_df.month > 203]


    
for col in ['p_type_88']:
    in_df = g_df3[g_df3[col] != 0].reset_index()
    
    in_df[col] = in_df[col].replace(-1,0)
    
    
    print(in_df.shape)
    Y_i = in_df.count_per_home.values
    df_EVAL = in_df.drop(columns = ['month','zipcode','home_count','i_count', 'count_per_home'])


    print(col + ' started')
    try:
        D_i = df_EVAL[col].values
        df_t = df_EVAL[['NREL_PSM_2019', 'cost_electricity','price']].to_numpy()
        model = CausalModel(Y = Y_i, #outcome 
                            D = D_i, #treatment
                            X = df_t)
        # added columns = ,'med_income','med_home_value','pop_density'
        #from hummingbird.ml import convert
        #model = convert(ci_model, 'pytorch')
        print('model complete')
        model.est_propensity() 
        
        print('propensity complete')
        model.est_via_matching()  
        print('matching complete')
        #model.estimates

        ate = round(model.estimates['matching']['ate'],5)
        
        treatment_dict[col] = ate
    except Exception as e:
        print(e)
        print(col, df_EVAL[df_EVAL[col] == 1].shape, df_EVAL[df_EVAL[col] == 0].shape)
        treatment_dict[col] = 'unknown'
    print(col + ' complete')

(111793, 26)
p_type_88 started
model complete
propensity complete
matching complete
p_type_88 complete


In [50]:
#treatment_dict
with open('../data/program_ate.p', 'wb') as fp:
    pickle.dump(treatment_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

Select zipcode
Select policy type
Select implementation date

Marks other zipcodes (chloropleth?)
Eliminates other zipcodes with policy
Identifies closest remaining zipcode (matching)
mean before
mean after
is slope consistent?


In [51]:
treatment_dict

{'p_type_31': 0.00154,
 'p_type_32': 0.0001,
 'p_type_33': 'unknown',
 'p_type_37': 1e-05,
 'p_type_46': -1e-05,
 'p_type_50': 'unknown',
 'p_type_78': 5e-05,
 'p_type_87': 5e-05,
 'p_type_92': 0.00042,
 'p_type_86': 0.00018,
 'p_type_88': -0.00023}

In [52]:
cols_rebate = ['p_type_87', 'p_type_88']

rebate_dict = {}

for col in cols_rebate:
    in_df = g_df[g_df[col] >= 0].reset_index()
    rebate_dict[col] = in_df[col].corr(in_df.discount)

In [53]:
#make violin plot of install distribution

,month,zipcode,i_count,size,price,discount,p_type_31,p_type_32,p_type_33,p_type_37,...,p_type_92,p_type_86,med_income,med_home_value,home_count,pop_density,NREL_PSM_2019,cost_electricity,count_per_home,rolling_slope


In [58]:
import statsmodels.formula.api as smf


reg = smf.ols('count_per_home ~ NREL_PSM_2019 + cost_electricity + price + p_type_31 + p_type_32 + p_type_37 + p_type_46 + p_type_78+ p_type_87+ p_type_88 + p_type_92 + p_type_86', data = g_df).fit()

ols_robust2_1 = reg.get_robustcov_results(cov_type= 'HC1')

print(ols_robust2_1.summary())

                            OLS Regression Results                            
Dep. Variable:         count_per_home   R-squared:                       0.059
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     4606.
Date:                Thu, 12 Aug 2021   Prob (F-statistic):               0.00
Time:                        09:26:44   Log-Likelihood:             1.4384e+07
No. Observations:             2218896   AIC:                        -2.877e+07
Df Residuals:                 2218883   BIC:                        -2.877e+07
Df Model:                          12                                         
Covariance Type:                  HC1                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -0.0003    2.4e-06  